In [70]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from random import randint
from time import sleep
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

In [2]:
review = pd.read_csv('../data/year_2018.csv')

In [3]:
review.drop('Unnamed: 0',axis = 1, inplace = True)

In [4]:
review.shape

(377430, 9)

In [5]:
review.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewText,summary
0,5.0,NaN,True,2018-10-04,A2IXGBWKX73O21,B01HI7D4VY,NaN,"quality, very bright","for the price, excellent"
1,5.0,NaN,True,2018-10-04,AHEX3FYTJ2AX6,B01HH36F74,{'Color:': ' CA100'},Works as expected.,Works as expected.
2,3.0,NaN,False,2018-10-03,A2J77LP43D7QMX,B01HF0YGCK,{'Color:': ' Black'},For a 3 pack they had a great price when I bou...,Great for the price
3,5.0,NaN,True,2018-10-03,A32FN9RNY65CKH,B01HGF6XTI,{'Size:': ' Charger'},Went to Boston and forgot my charger.\nOrdered...,Worked fine - got me through in a pinch
4,5.0,NaN,True,2018-10-02,A2ZE26LV7HVSAD,B01GUITK24,{'Style Name:': ' Bookshelf Speaker Mount 2 Pa...,I had to use my own drywall anchors which were...,Solid and kind of heavy


In [6]:
asin_list = list(review['asin'].unique())

In [7]:
len(asin_list)

56675

In [8]:
len(list(review['reviewerID'].unique()))

168153

In [77]:
# driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")
# product_list = []
# driver.get('https://www.amazon.com/gp/product/B01GZRIJRC')
# soup = BeautifulSoup(driver.page_source, 'lxml')
# product_info = {}
# ul_d = soup.find('ul', {'class': 'a-unordered-list a-vertical a-spacing-none'})
# product_info['category'] = [litag.text.strip() for litag in ul_c.find_all('li') if litag.text.strip() != '›']

In [108]:
#chrome_options = Options()
#chrome_options.add_argument('--dns-prefetch-disable')
#webdriver= Chrome(chrome_options = chrome_options)
driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")
product_list = []
for i in asin_list[1001:2000]:
    try:
        driver.get(f'https://www.amazon.com/gp/product/{i}')
    except:
        driver.close()
        driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")
        driver.get(f'https://www.amazon.com/gp/product/{i}')
    soup = BeautifulSoup(driver.page_source, 'lxml')
    product_info = {}
    try:
        product_info['asin'] = i
        product_info['product_name'] = soup.find('span', {'id':'productTitle'}).text.strip()
        product_info['product_rating'] = soup.find('span', {'class':'a-icon-alt'}).text.strip()
        product_info['product_rate_count'] = soup.find('span', {'id':'acrCustomerReviewText'}).text.strip()
        ul_c= soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'})
        product_info['category'] = [litag.text.strip() for litag in ul_c.find_all('li') if litag.text.strip() != '›']
        ul_d = soup.find('ul', {'class': 'a-unordered-list a-vertical a-spacing-none'})
        product_info['description'] = [litag.text.strip() for litag in ul_d.find_all('li')]
        product_list.append(product_info)
    except:
        product_info['asin'] = i
        product_info['product_name'] = 0
        product_info['product_rating'] = 0
        product_info['product_rate_count'] = 0
        product_info['category'] = 0
        product_info['description'] = 0
        
        product_list.append(product_info)
    sleep(randint(1,2))
product = pd.DataFrame(product_list)

In [109]:
product

,asin,product_name,product_rating,product_rate_count,category,description
0,B01B5RJ0XY,iExcell 100 Pcs Black 9 mm Cable Tie Base Sadd...,4.5 out of 5 stars,24 ratings,"[Electronics, Accessories & Supplies, Cord Man...",[✔ Product Name : Cable Tie Mount;Color : Blac...
1,B01FUYWMII,"USB Type-c Adapter, Acessorz high Speed USB Ty...",5.0 out of 5 stars,2 ratings,"[Electronics, Computers & Accessories, Compute...",[Make sure this fits\nby entering your model n...
2,B01HBSB7XC,Nalpar 6 x 10 feet Green Chromakey Backdrop Ba...,4.4 out of 5 stars,72 ratings,"[Electronics, Camera & Photo, Lighting & Studi...",[Make sure this fits\nby entering your model n...
3,B01FV2HJFU,"Aduro Tablet Gooseneck Universal Mount, Solid ...",3.8 out of 5 stars,638 ratings,"[Electronics, Computers & Accessories, Tablet ...",[Make sure this fits\nby entering your model n...
4,B01CS5O9YC,RCA to HDMI Video Converter - Tendak AV CVBS C...,3.7 out of 5 stars,16 ratings,"[Electronics, Accessories & Supplies, Audio & ...",[RCA/ AV to HDMI Converter will converts compo...
...,...,...,...,...,...,...
994,B01B289BCQ,Soot Sprites Susuwatari My Neighbor Totoro And...,4.4 out of 5 stars,35 ratings,"[Electronics, Computers & Accessories, Laptop ...",[Make sure this fits\nby entering your model n...
995,B017THGEFQ,Syrp Genie Mini Camera Motion Control - Wirele...,4.6 out of 5 stars,40 ratings,"[Electronics, Camera & Photo, Accessories, Tri...",[Make sure this fits\nby entering your model n...
996,B01CO4Y8IE,"Amazon Protective Cover for Kindle Paperwhite,...",4.2 out of 5 stars,742 ratings,"[Electronics, eBook Readers & Accessories, Cov...",[Designed by Amazon to perfectly fit your Kind...
997,B018PU87TM,"SIME-ON: Phone and Tablet Stand, Desk Holder C...",4.5 out of 5 stars,702 ratings,"[Electronics, Computers & Accessories, Tablet ...",[Universal Stand Compatibility: This universal...


In [110]:
product.to_csv('../data/product1001_2000.csv')